In [53]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px


import numpy as np

from freq_stacking_LFM_ACF_utils import *

In [54]:
# parameters
# min and max RF center freqs in Hz
fc_min = 500e6
fc_max = 1.48e9

delta_coeff = 0.01

# min and max Bws in Hz
Bmin = 10e6
Bmax = 20e6
delta_B = 1e6


max_delay_time = 5e-6
delay_time = 0.0

# chirp rate
chirp_rate = 50e6/10e-6


num_subpulses = 50

# maximum episode length
max_episode_timesteps = 3000

training_num = 200 # how many trajectories 


In [55]:
# Derived parameters
Fs_baseband = Bmax  # baseband samp freq in Hz
Fs_SWW_max = fc_max + Bmax / 2 - (fc_min - Bmax / 2)

# time window size
max_subpulse_duration = Bmax / chirp_rate
time_window_size = 10 * max_subpulse_duration + max_delay_time


# compute the state vector size
LFM_rx_subpulses, LFM_ref_subpulses = generate_LFM_rx_ref_subpulses_for_ACF(
    BW_RF_array=np.array([20e6]),
    chirp_rate=chirp_rate,
    time_window_size=time_window_size,
    Fs_baseband=Fs_baseband
)

N_max = compute_Nup_f(LFM_rx_subpulses[0], Fs_baseband, Fs_SWW_max) # the state vector size


In [56]:
def compute_D_f_d_t(fc_RF_freqs, BW_RF_array, weights):

    LFM_rx_subpulses, LFM_ref_subpulses = generate_LFM_rx_ref_subpulses_for_ACF(
        BW_RF_array, chirp_rate, time_window_size, Fs_baseband
    )

    # filter BWs
    Bs_array = BW_RF_array
    D_f, d_t = freq_stacking_v2(
        LFM_rx_subpulses,
        LFM_ref_subpulses,
        fc_RF_freqs,
        BW_RF_array,
        Bs_array,
        Fs_baseband,
        weights,
    )

    # compute times and freqs
    f_RF_sww_center, BW_RF_sww = compute_para_for_RF_sww(fc_RF_freqs, BW_RF_array)
    sww_samp_freq = BW_RF_sww

    performance = -int_sidelobe_ratio(np.abs(d_t))
    times = fftshift(np.fft.fftfreq(n=D_f.size, d = sww_samp_freq) * D_f.size)

    freqs = fftshift(np.fft.fftfreq(n=D_f.size, d = 1/sww_samp_freq))

    return freqs, D_f, times, d_t, performance


def normalized_Df_dt_in_dB(fc_RF_freqs, BW_RF_array, weights):

    freqs, D_f, times, d_t, performance = compute_D_f_d_t(fc_RF_freqs, BW_RF_array, weights)
    
    D_f[D_f == 0] = 1e-16 # avoid divide by zeros
    norm_fac = np.max( np.abs(D_f) )
    D_f_in_db = 20*np.log10(np.abs(D_f) / norm_fac)


    d_t[d_t == 0] = 1e-16 # to avoid divide by zero errors
    norm_fac = np.max(np.abs(d_t))  
    d_t_in_db =  20*np.log10( np.abs(d_t)  / norm_fac)

    return freqs, D_f_in_db, times, d_t_in_db, performance
    


In [57]:
# cener freqs are all the same
fc_RF_freqs = fc_min + np.arange(num_subpulses) * Bmin

- center freqs: uniformly spaced with 10MHz, 
- BW: all 10MHz, 
- Weights: all 1's, 
- (spectrum: non-overlapping spectrums)

In [58]:

BWs = np.linspace(Bmin, Bmax, 100)
performance_list = []

for bw in BWs:
    fc_RF_freqs = fc_min + np.arange(num_subpulses) * Bmin
    BW_RF_array = bw * np.ones_like(fc_RF_freqs)
    weights = np.ones(num_subpulses)

    freqs1, D_f_in_db1, times1, d_t_in_db1, performance1 = normalized_Df_dt_in_dB(
        fc_RF_freqs, BW_RF_array, weights
    )
    performance_list.append(performance1)


In [59]:
# plot time
fig1 = go.Figure()  # create a figure
fig1.add_trace(
    go.Scatter(x= BWs, y=performance_list, mode="lines", name=f"ISLR = {performance1}")
)  # the name attribute is the legend


